In [1]:
!wget https://www.dropbox.com/s/pdhwlpi2yeie0ol/movie-dataset.zip

--2022-08-04 22:51:21--  https://www.dropbox.com/s/pdhwlpi2yeie0ol/movie-dataset.zip
Resolving www.dropbox.com (www.dropbox.com)... 162.125.85.18, 2620:100:6017:18::a27d:212
Connecting to www.dropbox.com (www.dropbox.com)|162.125.85.18|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/pdhwlpi2yeie0ol/movie-dataset.zip [following]
--2022-08-04 22:51:22--  https://www.dropbox.com/s/raw/pdhwlpi2yeie0ol/movie-dataset.zip
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc0cedfbbe9674b565e6200add86.dl.dropboxusercontent.com/cd/0/inline/BqaCujSOcsA9n69_UevoWGObCk4ugJ330GKuK4_CxUbtTCOulMNXvP6NHIL7DjSMyiLk-kxp_nHGZxBBc-TwmUphwHuRINnWR6ES9zf6Ugpv4vblajSAyLeS8zjfcdsA0XUKTyErBtJUfKUju7FngNhHOijaBGLjPznsJ7YZvAK2Fw/file# [following]
--2022-08-04 22:51:22--  https://uc0cedfbbe9674b565e6200add86.dl.dropboxusercontent.com/cd/0/inline/BqaCujSOcsA9n69_UevoWGObCk4ugJ330GKuK4_CxUbtTCOulMN

In [5]:
!unzip -q 'movie-dataset.zip'

In [6]:
from tensorflow.keras.preprocessing import text_dataset_from_directory
from tensorflow.strings import regex_replace
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization
from tensorflow.keras.models import Sequential
from tensorflow.keras import Input
from tensorflow.keras.layers import Dense, RNN, SimpleRNNCell, Embedding, Dropout

In [7]:
def prepareData(dir):
  data = text_dataset_from_directory(dir)
  return data.map(
      lambda text, label: (regex_replace(text, '<br />',' '), label),)

In [8]:
train_data = prepareData('movie-reviews-dataset/train')
test_data = prepareData('movie-reviews-dataset/test')

for text_batch, label_batch in train_data.take(1):
  print(text_batch.numpy()[0])
  print(label_batch.numpy()[0])

Found 25000 files belonging to 2 classes.
Found 25000 files belonging to 2 classes.
b"Poor Ivy: Though to the manner born, she had the bad luck to marry a charming wastrel (Richard Ney). As the movie is set in the 20s or 30s, when rigid Victorian ideas of class were starting to fray at the edges, this uncertain status vexes her unduly. The Gretorexes (for so they are called) don't know where their next shilling is coming from but there are yachting parties and fancy-dress balls in posh pleasaunces aplenty to tempt her. When Ivy (Joan Fontaine) makes the acquaintance of a wealthy older gent (Herbert Marshall, who must have been born middle-aged), she sets one of her extravant chapeaux for him. Luckily, one of the beaux she still strings along (Patric Knowles) is a physician whose consulting rooms provide a cache of poison, with which she bids her hubby farewell. The fact that it implicates Knowles doesn't phase her a bit, even as the hours trickle by until he should be hanged by the nec

In [9]:
model = Sequential()
model.add(Input(shape=(1,),dtype='string'))

In [10]:
max_tokens = 1000
max_len = 100

vectorized_layer = TextVectorization(max_tokens = max_tokens,
                                     output_mode = 'int',
                                     output_sequence_length = max_len,)





In [13]:
train_text = train_data.map(lambda text, label : text)
vectorized_layer.adapt(train_text)

model.add(vectorized_layer)

In [14]:
# creating an embedding layer

model.add(Embedding(max_tokens + 1, 128))
rnn = RNN(SimpleRNNCell(64), return_sequences = False, return_state = False) # can change it from here
model.add(rnn)
model.add(Dense(64, activation = 'relu'))
model.add(Dense(1, activation = 'sigmoid'))

In [15]:
model.compile(loss = 'binary_crossentropy', optimizer = 'adam', metrics = ['accuracy'])

In [16]:
model.fit(train_data, epochs = 10)

Epoch 1/10
782/782 [==============================] - 32s 39ms/step - loss: 0.6947 - accuracy: 0.5199
Epoch 2/10
782/782 [==============================] - 31s 40ms/step - loss: 0.6706 - accuracy: 0.5822
Epoch 3/10
782/782 [==============================] - 30s 39ms/step - loss: 0.6377 - accuracy: 0.6365
Epoch 4/10
782/782 [==============================] - 32s 40ms/step - loss: 0.6430 - accuracy: 0.6244
Epoch 5/10
782/782 [==============================] - 31s 39ms/step - loss: 0.6683 - accuracy: 0.5905
Epoch 6/10
782/782 [==============================] - 31s 40ms/step - loss: 0.6687 - accuracy: 0.5816
Epoch 7/10
782/782 [==============================] - 30s 38ms/step - loss: 0.6787 - accuracy: 0.5613
Epoch 8/10
782/782 [==============================] - 30s 38ms/step - loss: 0.6423 - accuracy: 0.6406
Epoch 9/10
782/782 [==============================] - 31s 40ms/step - loss: 0.6347 - accuracy: 0.6511
Epoch 10/10
782/782 [==============================] - 30s 39ms/step - loss: 0.650

In [17]:
model.evaluate(test_data)

782/782 [==============================] - 13s 17ms/step - loss: 0.6350 - accuracy: 0.6505


[0.6349641680717468, 0.6505200266838074]

In [20]:
model.predict(['I loved the movie'])

array([[0.828953]], dtype=float32)